# LionAGI Introduction 2: Data Processing

LionAGI is very efficient and intuitive to handle data

In [14]:
from pathlib import Path
import lionagi as li

In [15]:
from timeit import default_timer as timer
start = timer()

In [16]:
ext=[".py", ".ipynb"]
data_dir = Path.cwd().parent / 'data'
project_name = "lionagi_intro"

### 1. Find Path

In [17]:
sources = li.dir_to_path(data_dir, ext, 
                         recursive=True, 
                         flat=False)

print(f"Found {len(sources[0])} files with extension '{ext[0]}' in source directory")
print(f"Found {len(sources[1])} files with extension '{ext[1]}' in source directory")
print(f"Found {len(li.to_list(sources, flat=True))} files in total in source directory")

Found 1673 files with extension '.py' in source directory
Found 375 files with extension '.ipynb' in source directory
Found 2048 files in total in source directory


### 2. Read Files

In [18]:
files = li.dir_to_files(data_dir, ext, 
                        recursive=True, 
                        project=project_name, to_csv=True) 

print(f"There are in total {sum(li.l_call(files, lambda x: x['file_size'])):,} chracters in {len(files)} non-empty files")

1998 logs saved to data/logs/sources/lionagi_intro_sources_2023-12-17T20_43_24_370487.csv
There are in total 47,482,838 chracters in 1998 non-empty files


In [19]:
test = files[50]

print(f"Files are read into {type(test)} type")
print(f"By default files include {test.keys()}\n\n-------------------------------------------------")
print(f"Sample from {li.to_list(sources, flat=True)[25]}\n-------------------------------------------------\n")
print(test['content'][:200])

Files are read into <class 'dict'> type
By default files include dict_keys(['project', 'folder', 'file', 'file_size', 'content'])

-------------------------------------------------
Sample from /Users/lion/Documents/GitHub/gitco/data/gitrepo/privateGPT-main/private_gpt/server/ingest/ingest_watcher.py
-------------------------------------------------

import argparse import sys from pathlib import Path from private_gpt.di import root_injector from private_gpt.server.ingest.ingest_service import IngestService from private_gpt.server.ingest.ingest_wa


In [20]:
lens = li.l_call(files, lambda x: len(x['content']))
min_, max_, avg_ = min(lens), max(lens), sum(lens)/len(lens)

print(f"Minimum length of files is {min_} in characters")
print(f"Maximum length of files is {max_:,} in characters")
print(f"Average length of files is {int(avg_):,} in characters")

print("""
the files seem to be fairly uneven in terms of length
which could bring problems in our subsequent analysis, we can stardardize them into chunks 
one convinient way to do this is via file_to_chunks function, it breaks the files into organized chunks
""")

Minimum length of files is 13 in characters
Maximum length of files is 11,639,375 in characters
Average length of files is 23,765 in characters

the files seem to be fairly uneven in terms of length
which could bring problems in our subsequent analysis, we can stardardize them into chunks 
one convinient way to do this is via file_to_chunks function, it breaks the files into organized chunks



### 3. Split to chunks

In [21]:
chunks = li.file_to_chunks(files, 
                            chunk_size=1000,  
                            overlap=0.2, 
                            threshold=200, to_csv=True, project=project_name, filename=f"{project_name}_chunks.csv")

48167 logs saved to data/logs/sources/lionagi_intro_chunks_2023-12-17T20_43_24_856171.csv


In [22]:
lens = li.l_call(li.to_list(chunks, flat=True), lambda x: len(x["chunk_content"]))
min_, max_, avg_ = min(lens), max(lens), sum(lens)/len(lens)

print(f"There are in total {len(li.to_list(chunks,flat=True)):,} chunks")
print(f"Minimum length of content in chunk is {min_} characters")
print(f"Maximum length of content in chunk is {max_:,} characters")
print(f"Average length of content in chunk is {int(avg_):,} characters")
print(f"There are in total {sum(li.l_call(chunks, lambda x: x['chunk_size'])):,} chracters in total")

print("""
Though the chunk_size is set to be 1000, the actual chunk_size depends on a number of factors:
- if the file is originally shorter than 1000, we will keep whole file as a chunk
- for files longer than 1000, we will break them into chunks with 1000 characters
    - we add overlap for each chunk with its neighbor. For example, if
        - first chunk has one neighbor, it would be 1000 + 1000 * 0.2/2 = 1100
        - second chunk has two neighbors, so it will be 1000 + 1000 * 0.2 = 1200
    - last chunk if longer than threshold , it will be 1000*0.2/2 + remaining length
    - if the remaining length is shorter than threshold, we will merge it with the preceeding chunk
""")

There are in total 48,167 chunks
Minimum length of content in chunk is 13 characters
Maximum length of content in chunk is 1,300 characters
Average length of content in chunk is 1,177 characters
There are in total 56,716,638 chracters in total

Though the chunk_size is set to be 1000 in this case, the actual chunk_size depends on a number of factors:
- if the file is originally shorter than 1000, we will keep whole file as a chunk
- we will chunk the files by 1000 characters, additionally
    - we add overlap for each chunk with neighbor. For example, if
        - first chunk would have one side of neighbor, it will be 1000 + 1000 * 0.2/2 = 1100
        - second chunk would have two sides of neighbor, it will be 1000 + 1000 * 0.2 = 1200
    - last chunk if longer than threshold, it will be 1000*0.2/2 + remaining length
    - if the remaining length is shorter than threshold, we will merge it with the preceeding chunk



### 4. Aggregate into bins

In [23]:
print("""Let's say you conducted certain llm analysis or similar data transformation on the chunks
and you now you want to put them in groups(bins) of certain range of length """)


Let's say you conducted certain llm analysis or similar data transformation on the chunks
and you now you want to put them in groups(bins) of certain range of length 


In [24]:
f = lambda x: x['chunk_content']
inputs = li.to_list(li.l_call(chunks, f))

bins = li.get_bins(inputs, upper=8000)
print(f"For bin size of 8000: There are in total {len(bins)} bins")

bins = li.get_bins(inputs, upper=4000)
print(f"For bin size of 4000: There are in total {len(bins)} bins")

bins = li.get_bins(inputs, upper=2000)
print(f"For bin size of 2000: There are in total {len(bins)} bins")

For bin size of 8000: There are in total 7839 bins
For bin size of 4000: There are in total 15845 bins
For bin size of 2000: There are in total 46908 bins


In [25]:
elapse = timer() - start

In [26]:
print(f"Total files processed {len(files):,}, with {sum(li.l_call(files, lambda x: x['file_size'])):,} chracters of content in total")
print(f"Total chunks produced {len(chunks):,}, with {sum(li.l_call(chunks, lambda x: x['chunk_size'])):,} chracters of content in total")

print(f"Total runtime: {elapse:.03f} seconds")

Total files processed 1,998, with 47,482,838 chracters of content in total
Total chunks produced 48,167, with 56,716,638 chracters of content in total
Total runtime: 1.467 seconds


In [ ]:
print("""
chunks vs bins
depends on use case and nature of data:
- input type
    - chunk_text works with a single input string
    - get_bins works with a list of items, not just text
- output type
    - chunk_text returns a list of text chunks
    - get_bins returns a list of list where each inner list contains indices of items forming a bin
- use case considerations
    - chunk_text for text segmentation and analysis, especially when dealing with a single large text
    - get_bins when you have a list of items and want to group them into bins based on their cumulative length    
""")